#### Quake Heap Implementation

Given a set of items $P$ in which each item has a priority, the `quake-heap` on $P$ is defined as a `tournament forest` $\tau = \set{T_1, T_2, ...,T_L}$ where $T_i$ is a  (binary) tournament tree on $P_i \subset P$ such that $P_i \cap P_j = \empty$ if $i \neq j$.

The leaf-nodes in the forest are the items from $P$. All leaf nodes are maintained at the same height, which is the lowest level in the forest. We define $n_i$ as the number of nodes which have height $i$. We also maintain the following `invariant`: $n_{i+1} \leq \alpha n_i$ for $i=1,2,..,h_{max}-1$, where $\alpha \in (\frac{1}{2},1)$ is a fixed constant. 

The quake-heap data structure supports the following operations:

* `Link(T_1,T_2)` - creates a new node $u$ with $priority(u) = min(priority(root(T_1)), priority(root(T_2)))$, then attaches $T_1$ and $T_2$ as subtrees of u to form a new tree $T'$

* `cut(u)` -  given a node `u` whose priority is larger than that of it's parent $v$, this operation detaches u from it's parent to create a new tree rooted at $u$. 

* `Insert(u)` - inserts a new item $u$ into the quake-heap, this creates a new tree containing only $u$

* `Decrease-Min(x,k)` - first cut(u) is invoked, where u is the highest node containing the priority of item $x$, then the priority of item $x$ is changed to $k$. 

* `Delete-Min()`- deletes the item $x$ with smallest priority from the quake-heap, this is done in three steps:

    (i) remove all nodes in the $u$-$x$ path, where u is the highest node containing the priority of item $x$, we do this by performing a sequence of cut operation on every node in the path starting with the child of $u$ and after each cut, we delete the parent of the corresponding node on which we performed the cut. Finally when we reach the leaf and delete it's parent, we also delete the leaf.
    
    (ii) Link all pairs of trees which are at the same height

    (iii) Maintain number of nodes at height invariant property by finding the smallest height $h$ at which a violation occurs, i.e. the smallest $h$ for which $n_{h+1} > \alpha n_h$, and delete all nodes above this height. 

To facilitate the quake-heap operations, each node in the forest will have some auxiliary pointers:

* Each leaf node has a pointer to the highest internal node containing it's priority. 

* Each internal node has a pointer to the leaf node whose priority it contains.

To facilitate linking of trees in particular, we will also maintain a dictionary $A$ which maps height $i$ to a list of trees which have that height. We also maintain a list $L$ which contains all the heights for which at least two trees have that height, which allows us to quickly find all the heights at which we have pairs of trees that may be linked and retreive those trees. 

In [2]:
from collections import defaultdict
from graphviz import Digraph, Source

def add_nodes_edges(graph, node, counter=0):
    if node is not None:
        root = str(counter)
        graph.node(root, str(node.priority))
        if node.left is not None:
            left = str(counter + 1)
            graph.edge(root, left)
            counter = add_nodes_edges(graph, node.left, counter + 1)
        if node.right is not None:
            right = str(counter + 1)
            graph.edge(root, right)
            counter = add_nodes_edges(graph, node.right, counter + 1)
    return counter

def visualize_tree(root):
    graph = Digraph()
    add_nodes_edges(graph, root)
    return Source(graph.source)  

In [3]:
# linked list node
class LLNode:
    def __init__(self, data=None, prev=None, next=None):
        self.data = data
        self.prev = prev
        self.next = next

# doubly linked list data structure
class DoubleLinkedList():
    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0

    def delete(self, node):
        if node == self.head:
            self.head = node.next
        if node == self.tail:
            self.tail = node.prev
        if node.prev is not None:
            node.prev.next = node.next
        if node.next is not None:
            node.next.prev = node.prev
        self.size -= 1    

    def insert(self, node):
        if self.head is None:
            self.head = node
            self.tail = node
        else:
            node.next = self.head
            self.head.prev = node
            self.head = node
        self.size += 1

# tournament tree node
class TreeNode:
    def __init__(self, priority=None, data=None, left=None, right=None, aux_leaf=None, aux_highest=None):
        self.priority = priority
        self.data = data
        self.left = left
        self.right = right
        self.aux_leaf = aux_leaf        # auxiliary pointer to leaf node 
        self.aux_highest = aux_highest  # auxiliary pointer to highest internal node


# quake heap data structure
class QuakeHeap():
    def __init__(self):
        self.forest = []  # list of root nodes of all tournament trees in the forest
        self.size = 0 # number of item in forest      
        self.A = defaultdict(list)   # dictionary mapping height to a tuple: (list of trees of that height, pointer to that height in the doubly-linked list of heights with linkable trees if there are at least two trees of that height in the forest)
        self.L = DoubleLinkedList    # doubly-linked list of heights for which at least two trees have that height (i.e. heights with linkable trees)

    def insert(self, priority, data=None):
        """
        Insert a new node with given priority and data into the heap
        """
        new_node = TreeNode(priority=priority, data=data)
        self.forest.append(new_node)
        self.size += 1
        if 1 in self.A:
            self.A[1][0].append(new_node)
            if len(self.A[1][0]) == 2:
                # insert this height into the doubly-linked list of heights with linkable trees, only need to insert once when the number of trees becomes 2
                height_node = LLNode(data=1)
                self.L.insert(height_node)  
                self.A[1][1] = height_node
        else:
            self.A[1] = ([new_node], None)
        self.L.sort()

    """
        T1 : root node of first tree
        T2 : root node of second tree 
    """
    def link(self, T1, T2):
        # first remove T1 and T2 from the forest
        self.forest.remove(T1)
        self.forest.remove(T2)
        # create a new root node and attach T1 and T2 as it's subtrees
        new_root = TreeNode()
        new_root.left = T1
        new_root.right = T2
        # find which subtree has the lower priority and get it's auxiliary pointer to it's leaf node
        aux_leaf = T1.aux_leaf if T1.aux_leaf.priority < T2.aux_leaf.priority else T2.aux_leaf 
        # create pointers to and from the new root node and the auxiliary leaf node
        new_root.aux_leaf = aux_leaf
        aux_leaf.aux_highest = new_root

        # add the new root node to the forest
        self.forest.append(new_root)